**building a chat model**

In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gemma:2b")

model

ChatOllama(model='gemma:2b')

***1. Building a Simple Chain***

In [3]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate(
    template="""
        Write a detailed article on th topic: {topic}
    """,
    input_variables=['topic']
)

template

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        Write a detailed article on th topic: {topic}\n    ')

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser

StrOutputParser()

In [5]:
chain = template | model | parser


In [6]:
user_input = input("Enter the topic...")


Enter the topic... Himalayas in India


In [7]:
temp = template.invoke({'topic':user_input})
temp

StringPromptValue(text='\n        Write a detailed article on th topic: Himalayas in India\n    ')

In [8]:

response = chain.invoke({'topic':user_input})

print(response)

## The Himalayas in India: A Mountain Paradise

The Himalayas, located in the western Himalayas of India, are a breathtaking landscape of towering peaks, deep valleys, and shimmering lakes. These majestic mountains are not only a natural wonder but also a spiritual haven, attracting pilgrims from all corners of the world. 

**The Himalayas in a Nutshell:**

* **Area:** 8,000 sq. km
* **Mountains:** 8,602 peaks, with the highest being Mount Everest at 8,848.86 m
* **Rivers:** The Ganges, Indus, and Brahmaputra rivers run through the Himalayas, forming a natural boundary between India and Nepal
* **Flora and Fauna:** Diverse ecosystem with a rich variety of flora and fauna, including endangered species like the Himalayan black-buck and the alpine golden-eagle.
* **Climate:** Subtropical highland climate with a distinct winter and summer season.
* **Culture and Heritage:** The Himalayas are rich in ancient traditions and cultures, with numerous monasteries, temples, and festivals celebrat

In [9]:
# pip install grandalf

In [10]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOllama |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


***2. Building Sequential Chain***

In [11]:
template_1 = PromptTemplate(
    template = """
        Write a detailed blog on the topic of : {topic}. 
        Mention all the relevant facts and figures backed by a reputed organization.
        Also, be sure to cover a example or short story with fictional character related to such concept or topic!
    """,
    input_variables = ['topic']
)

template_1

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        Write a detailed blog on the topic of : {topic}. \n        Mention all the relevant facts and figures backed by a reputed organization.\n        Also, be sure to cover a example or short story with fictional character related to such concept or topic!\n    ')

In [13]:
template_2 = PromptTemplate(
    template = """
        the article is : {detailed_article}.
        Summarize all article in 3 to 4 lines in simple words by keeping important points.
    """,
    input_variables = ['detailed_article']
)

template_2

PromptTemplate(input_variables=['detailed_article'], input_types={}, partial_variables={}, template='\n        the article is : {detailed_article}.\n        Summarize all article in 3 to 4 lines in simple words by keeping important points.\n    ')

In [15]:
chain_2 = template_1 | model | parser | template_2 | model | parser


In [16]:
response = chain_2.invoke({'topic':user_input})

print(response)

Sure, here's a summary of the article in 3 to 4 lines:

The Himalayas are the highest mountain range on Earth, with Mount Everest standing tall at 8,848.86 m. They offer breathtaking beauty, adventure sports, and a rich cultural heritage. However, they also present significant challenges, including high altitude, environmental degradation, and climate change.


In [17]:
chain_2.get_graph().print_ascii()


     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOllama |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

***3. Building Parallel Chain using - Runnables***

In [18]:
template_1 = PromptTemplate(
    template = """
        hey! help me to write a proper detailed notes on the topic of : {topic}.
        cover content in easy simple language.
        write major facts and figures with a proper organization report or findings to support the data.
        keep the content structured in points form, instead of a paragraph!
    """,
    input_variables = ['topic']
)
template_1

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        hey! help me to write a proper detailed notes on the topic of : {topic}.\n        cover content in easy simple language.\n        write major facts and figures with a proper organization report or findings to support the data.\n        keep the content structured in points form, instead of a paragraph!\n    ')

In [19]:
template_2 = PromptTemplate(
    template = """
        hey! help me to write 5 quiz questions on the topic of : {topic}
        keep the simple level. 
    """,
    input_variables = ['topic']
)

template_2

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        hey! help me to write 5 quiz questions on the topic of : {topic}\n        keep the simple level. \n    ')

In [20]:
template_3 = PromptTemplate(
    template = """
        hey! help me to merge the content of notes and quizzes in a simple and single way!
        the notes are -> {notes} and, the quizzes ques. are -> {quiz}
    """,
    input_variables = ['notes', 'quiz']
)

template_3

PromptTemplate(input_variables=['notes', 'quiz'], input_types={}, partial_variables={}, template='\n        hey! help me to merge the content of notes and quizzes in a simple and single way!\n        the notes are -> {notes} and, the quizzes ques. are -> {quiz}\n    ')

In [24]:
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel({
    "notes": template_1 | model | parser,
    "quiz": template_2 | model | parser
})

parallel_chain


{
  notes: PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        hey! help me to write a proper detailed notes on the topic of : {topic}.\n        cover content in easy simple language.\n        write major facts and figures with a proper organization report or findings to support the data.\n        keep the content structured in points form, instead of a paragraph!\n    ')
         | ChatOllama(model='gemma:2b')
         | StrOutputParser(),
  quiz: PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        hey! help me to write a proper detailed notes on the topic of : {topic}.\n        cover content in easy simple language.\n        write major facts and figures with a proper organization report or findings to support the data.\n        keep the content structured in points form, instead of a paragraph!\n    ')
        | ChatOllama(model='gemma:2b')
        | StrOutputParser()
}

In [25]:
merge_chain = template_3 | model | parser
merge_chain

PromptTemplate(input_variables=['notes', 'quiz'], input_types={}, partial_variables={}, template='\n        hey! help me to merge the content of notes and quizzes in a simple and single way!\n        the notes are -> {notes} and, the quizzes ques. are -> {quiz}\n    ')
| ChatOllama(model='gemma:2b')
| StrOutputParser()

In [27]:
chain_3 = parallel_chain | merge_chain


In [28]:
response = chain_3.invoke({'topic':user_input})

print(response)

**Merged Content:**

The Himalayas, a majestic mountain range that stretches across the northern border of India, are a breathtaking landscape of towering peaks, deep valleys, and stunning scenery. With its rich biodiversity, rich culture, and breathtaking landscapes, the Himalayas offer an unparalleled experience that will leave you breathless.

**Key Facts and Figures:**

* **Height:** The Himalayas are the highest mountain range on Earth, with Mount Everest being the highest peak at 8,848.86 m above sea level.
* **Area:** The Himalayas cover a massive 3,287 square kilometers, making up 7.6% of the country's land area.
* **Elevation:** The range stretches from 320 km in the west to 1,700 km in the east.
* **Mountains:** The Himalayas are home to the highest peaks on Earth, including Mount Everest, K2, Kangchenjunga, and Makalu.
* **Rivers:** The range boasts the longest rivers in the world, including the Ganges, Brahmaputra, and Indus, which originate in the Himalayas.
* **Biodiversi

In [29]:
chain_3.get_graph().print_ascii()


            +---------------------------+            
            | Parallel<notes,quiz>Input |            
            +---------------------------+            
                 **               **                 
              ***                   ***              
            **                         **            
+----------------+                +----------------+ 
| PromptTemplate |                | PromptTemplate | 
+----------------+                +----------------+ 
          *                               *          
          *                               *          
          *                               *          
  +------------+                    +------------+   
  | ChatOllama |                    | ChatOllama |   
  +------------+                    +------------+   
          *                               *          
          *                               *          
          *                               *          
+-----------------+         

***4. Building a Feedback Loop***

In [30]:
answer_prompt = PromptTemplate(
    template="Answer the question clearly:\nQuestion: {question}",
    input_variables=["question"]
)

improve_prompt = PromptTemplate(
    template="""
The previous answer was rated as bad.
Improve it based on the feedback below.

Answer:
{answer}

Feedback:
{feedback}

Provide a better answer.
""",
    input_variables=["answer", "feedback"]
)


In [32]:

from langchain_core.runnables import RunnableBranch

# Chains
# ------------------------
answer_chain = answer_prompt | model

improve_chain = improve_prompt | model

# ------------------------
# Conditional Logic
# ------------------------
feedback_chain = RunnableBranch(
    # If feedback contains "bad" or "negative"
    (
        lambda x: "bad" in x["feedback"].lower()
        or "negative" in x["feedback"].lower(),
        improve_chain
    ),
    # Else (default path)
    lambda x: x["answer"]
)

In [33]:
# Run Example
# ------------------------
question = "What is machine learning?"
feedback = "This answer is bad and unclear"

In [35]:
# Step 1: Generate answer
answer = answer_chain.invoke({"question": question}).content

In [ ]:

# Step 2: Apply feedback condition
final_response = feedback_chain.invoke({
    "answer": answer,
    "feedback": feedback
})

print("Final Response:\n", final_response)